<a href="https://colab.research.google.com/github/basimakram/synthetic-data-generation/blob/main/random_location_generator_within_coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Random Location Generator

This notebook generates random geographic points within specified boundary coordinates and optionally retrieves their addresses using Google Maps API.

## Features:
- Generate random points within polygon boundaries
- Support for coordinate lists and GeoJSON files
- Address lookup using Google Maps API
- Interactive map visualization with Folium

## Requirements:
- Google Maps API key (for address lookup)
- Required Python packages: `shapely`, `googlemaps`, `folium`

##Install Required Packages

Run this cell to install all necessary packages if they're not already installed.

In [1]:
# Install required packages
!pip install shapely googlemaps folium

print("All packages installed successfully!")

All packages installed successfully!


## Import Required Libraries and Set Google Maps API Key

In [2]:
import json
import random
import requests
from shapely.geometry import Polygon, Point
import googlemaps
import folium

print("All libraries imported successfully!")

All libraries imported successfully!


In [15]:
# Configuration variables
GOOGLE_API_KEY = "YOUR_GOOGLE_MAPS_API_KEY_HERE"  # Replace with your actual API key

print(f"Configuration set. API Key provided: {'Yes' if GOOGLE_API_KEY != 'YOUR_GOOGLE_MAPS_API_KEY_HERE' else 'No - Please update API key'}")

Configuration set. API Key provided: No - Please update API key


## Core Random Location Generator

In [4]:
def generate_random_locations(coordinates, num_points=10, google_api_key=None):
    """
    Generate random points within given coordinates

    Args:
        coordinates: List of {"latitude": float, "longitude": float} OR GeoJSON file path
        num_points: Number of random points to generate
        google_api_key: Google Maps API key for addresses. This field is optional and by default set to None :)

    """

    # Handle GeoJSON file input
    if isinstance(coordinates, str) and coordinates.endswith('.geojson'):
        print(f"Loading GeoJSON file: {coordinates}")
        with open(coordinates, 'r') as f:
            geojson = json.load(f)

        # Extract coordinates from GeoJSON
        if geojson['type'] == 'Feature':
            coords = geojson['geometry']['coordinates'][0]
        else:
            coords = geojson['coordinates'][0]

        # Convert to polygon coordinates (longitude, latitude)
        polygon_coords = [(coord[0], coord[1]) for coord in coords]
        print(f"Loaded {len(polygon_coords)} boundary points from GeoJSON")

    # Handle coordinate list input
    else:
        # Convert to polygon coordinates (longitude, latitude)
        polygon_coords = [(coord["longitude"], coord["latitude"]) for coord in coordinates]
        print(f"Using {len(polygon_coords)} boundary points from coordinate list")

    # Create polygon and get bounding box
    polygon = Polygon(polygon_coords)
    min_x, min_y, max_x, max_y = polygon.bounds
    print(f"Polygon bounds: ({min_x:.6f}, {min_y:.6f}) to ({max_x:.6f}, {max_y:.6f})")

    # Generate random points inside polygon using rejection sampling
    print(f"Generating {num_points} random points within polygon...")
    random_points = []
    attempts = 0

    while len(random_points) < num_points:
        # Generate random point within bounding box
        x = random.uniform(min_x, max_x)  # longitude
        y = random.uniform(min_y, max_y)  # latitude
        point = Point(x, y)
        attempts += 1

        # Check if point is inside the polygon
        if polygon.contains(point):
            random_points.append((x, y))  # (longitude, latitude)

    print(f"Generated {len(random_points)} points in {attempts} attempts")

    # Convert to final format and optionally get addresses
    locations = []
    gmaps = googlemaps.Client(key=google_api_key) if google_api_key else None

    if gmaps:
        print("Fetching addresses using Google Maps API...")

    for i, (lng, lat) in enumerate(random_points):
        location = {
            'latitude': lat,
            'longitude': lng
        }

        # Get address if API key provided
        if gmaps:
            try:
                result = gmaps.reverse_geocode((lat, lng))
                location['address'] = result[0]['formatted_address'] if result else "Address not found"
            except Exception as e:
                location['address'] = f"Error fetching address: {str(e)}"
                print(f"Warning: Could not fetch address for point {i+1}: {str(e)}")

        locations.append(location)

    print(f"Successfully generated {len(locations)} locations")
    return locations

## Core Visualization Function

In [5]:
def visualize_locations(boundary_coordinates, generated_locations,
                        output_file="locations_map.html", google_api_key=None,
                        geocode_timeout=5):
    """
    Create an interactive map showing boundary and generated points

    Args:
        boundary_coordinates: Original boundary coordinates OR GeoJSON file path
        generated_locations: Generated random locations (list of dicts with 'latitude' and 'longitude')
        output_file: HTML file to save the map
        google_api_key: (optional) Google Maps Geocoding API key for reverse geocoding
        geocode_timeout: HTTP timeout seconds for geocoding requests

    Returns:
        folium.Map: The created map object
    """
    print(f"Creating interactive map: {output_file}")

    # -- Helper: reverse geocode using Google Geocoding API --
    def _reverse_geocode(lat, lng):
        if not google_api_key:
            return None
        try:
            url = "https://maps.googleapis.com/maps/api/geocode/json"
            params = {"latlng": f"{lat},{lng}", "key": google_api_key}
            resp = requests.get(url, params=params, timeout=geocode_timeout)
            resp.raise_for_status()
            data = resp.json()
            if data.get("status") == "OK" and data.get("results"):
                return data["results"][0].get("formatted_address")
            else:
                # could log data.get("status") or data for debugging
                return None
        except Exception:
            # keep it simple: return None on any error (network, parsing, quota, etc.)
            return None

    # Handle GeoJSON file input
    if isinstance(boundary_coordinates, str) and boundary_coordinates.endswith('.geojson'):
        with open(boundary_coordinates, 'r') as f:
            geojson = json.load(f)

        if geojson.get('type') == 'Feature':
            coords = geojson['geometry']['coordinates'][0]
        else:
            coords = geojson['coordinates'][0]

        boundary_points = [[coord[1], coord[0]] for coord in coords]  # [lat, lng]
        center_lat = sum([point[0] for point in boundary_points]) / len(boundary_points)
        center_lng = sum([point[1] for point in boundary_points]) / len(boundary_points)

    else:
        boundary_points = [[coord["latitude"], coord["longitude"]] for coord in boundary_coordinates]
        lats = [coord["latitude"] for coord in boundary_coordinates]
        lngs = [coord["longitude"] for coord in boundary_coordinates]
        center_lat = sum(lats) / len(lats)
        center_lng = sum(lngs) / len(lngs)

    # If google_api_key provided, attempt to populate addresses for generated_locations
    if google_api_key:
        for loc in generated_locations:
            if 'address' not in loc or not loc['address']:
                addr = _reverse_geocode(loc['latitude'], loc['longitude'])
                if addr:
                    loc['address'] = addr

    # Create map centered on the boundary area
    m = folium.Map(location=[center_lat, center_lng], zoom_start=11, tiles='OpenStreetMap')

    # Add boundary polygon
    folium.Polygon(
        locations=boundary_points,
        popup="Boundary Area",
        tooltip="Search Boundary",
        color='red',
        weight=3,
        fillColor='red',
        fillOpacity=0.1
    ).add_to(m)

    # Add boundary points as small markers
    for i, point in enumerate(boundary_points):
        folium.CircleMarker(
            location=point,
            radius=5,
            popup=f"Boundary Point {i+1}<br>Lat: {point[0]:.6f}<br>Lng: {point[1]:.6f}",
            tooltip=f"Boundary {i+1}",
            color='red',
            fill=True,
            weight=2
        ).add_to(m)

    # Add generated points with detailed popups; tooltip will show address if present
    for i, location in enumerate(generated_locations, 1):
        popup_text = f"<b>{location.get('address','Generated Point '+str(i))}</b><br>"
        popup_text += f"Latitude: {location['latitude']:.6f}<br>"
        popup_text += f"Longitude: {location['longitude']:.6f}<br>"
        if 'address' in location:
            popup_text += f"Address: {location['address']}"

        # tooltip: prefer short address, otherwise "Point i"
        if 'address' in location and location['address']:
            tooltip_text = (location['address'][:80] + '…') if len(location['address']) > 80 else location['address']
        else:
            tooltip_text = f"Point {i}"

        folium.Marker(
            location=[location['latitude'], location['longitude']],
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=tooltip_text,
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # Add simplified legend (no external icons)
    legend_html = '''
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        min-width: 160px;
        max-width: 220px;
        background-color: white;
        border: 2px solid #888;
        z-index: 9999;
        font-size: 14px;
        padding: 8px;
        border-radius: 6px;
        box-shadow: 0 1px 4px rgba(0,0,0,0.3);
        ">
      <div style="font-weight:700; margin-bottom:6px;">Legend</div>
      <div style="display:flex; align-items:center; margin-bottom:4px;">
        <span style="display:inline-block; width:12px; height:12px; border-radius:6px; background:#d9534f; margin-right:8px;"></span>
        <span>Boundary</span>
      </div>
      <div style="display:flex; align-items:center;">
        <span style="display:inline-block; width:12px; height:12px; border-radius:6px; background:#337ab7; margin-right:8px;"></span>
        <span>Generated Points</span>
      </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # # Save map (kept commented as requested)
    # m.save(output_file)
    # print(f"Map saved as {output_file}")
    return m


## Sample Coordinates Example

These coordinates define a polygon boundary around Dallas, Texas.

In [6]:
# Dallas area boundary coordinates
dallas_coordinates = [
    {"latitude": 32.85848133654008, "longitude": -96.61047961349752},
    {"latitude": 32.874630023308306, "longitude": -96.68326403732564},
    {"latitude": 32.92651659291463, "longitude": -96.81509997482564},
    {"latitude": 32.906918572843665, "longitude": -96.90299059982564},
    {"latitude": 32.86540256238283, "longitude": -97.00186755295064},
    {"latitude": 32.76614674254919, "longitude": -97.02246691818502},
    {"latitude": 32.73958279699744, "longitude": -96.95380236740377},
    {"latitude": 32.70838892796183, "longitude": -96.90024401779439},
    {"latitude": 32.688742746104964, "longitude": -96.83157946701314},
    {"latitude": 32.700299847402476, "longitude": -96.74094225998189},
    {"latitude": 32.715321841911496, "longitude": -96.67914416427877},
    {"latitude": 32.744203182702414, "longitude": -96.62695910568502},
    {"latitude": 32.79732039308659, "longitude": -96.61047961349752},
    {"latitude": 32.85848133654008, "longitude": -96.61047961349752}  # Closing the polygon
]

print(f"Dallas boundary defined with {len(dallas_coordinates)} points")
print(f"Boundary area covers approximately: {dallas_coordinates[0]['latitude']:.3f}°N to {max(c['latitude'] for c in dallas_coordinates):.3f}°N")
print(f"                                    {min(c['longitude'] for c in dallas_coordinates):.3f}°W to {max(c['longitude'] for c in dallas_coordinates):.3f}°W")

Dallas boundary defined with 14 points
Boundary area covers approximately: 32.858°N to 32.927°N
                                    -97.022°W to -96.610°W


In [7]:
# Generate 5 random locations in Dallas area
print("=" * 50)
print("GENERATING DALLAS LOCATIONS")
print("=" * 50)

dallas_locations = generate_random_locations(
    dallas_coordinates,
    num_points=5,
    google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None
)

# Display results
print("\n📍 Generated Dallas locations:")
print("-" * 40)
for i, loc in enumerate(dallas_locations, 1):
    print(f"{i}. Latitude: {loc['latitude']:.6f}, Longitude: {loc['longitude']:.6f}")
    if 'address' in loc:
        print(f"   Address: {loc['address']}")
    print()

GENERATING DALLAS LOCATIONS
Using 14 boundary points from coordinate list
Polygon bounds: (-97.022467, 32.688743) to (-96.610480, 32.926517)
Generating 5 random points within polygon...
Generated 5 points in 9 attempts
Fetching addresses using Google Maps API...
Successfully generated 5 locations

📍 Generated Dallas locations:
----------------------------------------
1. Latitude: 32.821246, Longitude: -96.831003
   Address: 5426 Denton Dr, Dallas, TX 75235, USA

2. Latitude: 32.901235, Longitude: -96.785627
   Address: 7023 Midbury Dr, Dallas, TX 75230, USA

3. Latitude: 32.884255, Longitude: -96.886471
   Address: 11041 Harry Hines Blvd, Dallas, TX 75229, USA

4. Latitude: 32.848372, Longitude: -96.848009
   Address: R5X2+8Q Dallas, TX, USA

5. Latitude: 32.779960, Longitude: -96.634437
   Address: 2402 Monticello Dr, Mesquite, TX 75149, USA



In [8]:
# Create visualization for Dallas area
print("🗺️ Creating Dallas area visualization...")
dallas_map = visualize_locations(dallas_coordinates, dallas_locations, "dallas_locations.html", google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None)
print("✅ Dallas map created successfully!")

# Display the map in Jupyter notebook
dallas_map

🗺️ Creating Dallas area visualization...
Creating interactive map: dallas_locations.html
✅ Dallas map created successfully!


In [9]:
# Lahore area boundary coordinates
lahore_coordinates = [
    {"longitude": 74.28360882041864, "latitude": 31.503748742974174},
    {"longitude": 74.41705288271304, "latitude": 31.49107678116169},
    {"longitude": 74.41805938074691, "latitude": 31.457817119607554},
    {"longitude": 74.28594106642123, "latitude": 31.439284715710603}
]

print(f"Lahore boundary defined with {len(lahore_coordinates)} points")
print(f"Boundary area covers approximately: {min(c['latitude'] for c in lahore_coordinates):.3f}°N to {max(c['latitude'] for c in lahore_coordinates):.3f}°N")
print(f"                                    {min(c['longitude'] for c in lahore_coordinates):.3f}°E to {max(c['longitude'] for c in lahore_coordinates):.3f}°E")

Lahore boundary defined with 4 points
Boundary area covers approximately: 31.439°N to 31.504°N
                                    74.284°E to 74.418°E


In [10]:
# Generate 3 random locations in Lahore area
print("=" * 50)
print("GENERATING LAHORE LOCATIONS")
print("=" * 50)

lahore_locations = generate_random_locations(
    lahore_coordinates,
    num_points=10,
    google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None
)

# Display results
print("\n📍 Generated Lahore locations:")
print("-" * 40)
for i, loc in enumerate(lahore_locations, 1):
    print(f"{i}. Latitude: {loc['latitude']:.6f}, Longitude: {loc['longitude']:.6f}")
    if 'address' in loc:
        print(f"   Address: {loc['address']}")
    print()

GENERATING LAHORE LOCATIONS
Using 4 boundary points from coordinate list
Polygon bounds: (74.283609, 31.439285) to (74.418059, 31.503749)
Generating 10 random points within polygon...
Generated 10 points in 13 attempts
Fetching addresses using Google Maps API...
Successfully generated 10 locations

📍 Generated Lahore locations:
----------------------------------------
1. Latitude: 31.457987, Longitude: 74.395746
   Address: Plot L 570, Sector L Phase 5 D.H.A, Lahore, Pakistan

2. Latitude: 31.476541, Longitude: 74.328100
   Address: Plot 95 G, Block G Model Town, Lahore, Pakistan

3. Latitude: 31.484970, Longitude: 74.292426
   Address: F7PR+5WW, Wafaqi Colony, Lahore, Pakistan

4. Latitude: 31.483019, Longitude: 74.305206
   Address: 135, Block B Faisal Town, Lahore, Pakistan

5. Latitude: 31.457746, Longitude: 74.328114
   Address: 52A Quaid-E-Azam Boulevard, COLONY Quaid e Azam Industrial Estate, Lahore, Pakistan

6. Latitude: 31.455451, Longitude: 74.392155
   Address: Plot 172, HH

In [11]:
# Create visualization for Lahore area
print("🗺️ Creating Lahore area visualization...")
lahore_map = visualize_locations(lahore_coordinates, lahore_locations, "lahore_locations.html", google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None)
print("✅ Lahore map created successfully!")

# Display the map in Jupyter notebook
lahore_map

🗺️ Creating Lahore area visualization...
Creating interactive map: lahore_locations.html
✅ Lahore map created successfully!


## GeoJson File Example

This section demonstrates how to use GeoJSON files as input for boundary definitions.

In [12]:
addison_location = "addison.geojson"

# Generate locations from GeoJSON
addison_locations = generate_random_locations(
    addison_location,
    num_points=10,
    google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None
)

# Visualize GeoJSON locations
addison_map = visualize_locations(
    addison_location,
    addison_locations,
    google_api_key=GOOGLE_API_KEY if GOOGLE_API_KEY != "YOUR_GOOGLE_MAPS_API_KEY_HERE" else None
)



Loading GeoJSON file: addison.geojson
Loaded 47 boundary points from GeoJSON
Polygon bounds: (-96.914055, 32.915996) to (-96.741040, 33.039971)
Generating 10 random points within polygon...
Generated 10 points in 12 attempts
Fetching addresses using Google Maps API...
Successfully generated 10 locations
Creating interactive map: locations_map.html


In [13]:
# Display results
print("\n📍 Generated Addison locations:")
print("-" * 40)
for i, loc in enumerate(addison_locations, 1):
    print(f"{i}. Latitude: {loc['latitude']:.6f}, Longitude: {loc['longitude']:.6f}")
    if 'address' in loc:
        print(f"   Address: {loc['address']}")
    print()


📍 Generated Addison locations:
----------------------------------------
1. Latitude: 32.925581, Longitude: -96.826617
   Address: 4817 Thunder Rd, Dallas, TX 75244, USA

2. Latitude: 33.006459, Longitude: -96.796790
   Address: 19019 Preston Rd, Dallas, TX 75252, USA

3. Latitude: 32.971901, Longitude: -96.791427
   Address: 6604 La Manga Dr, Dallas, TX 75248, USA

4. Latitude: 32.927203, Longitude: -96.823292
   Address: 4901 Lyndon B Johnson Fwy Ste 400, Farmers Branch, TX 75244, USA

5. Latitude: 32.923697, Longitude: -96.846762
   Address: 4014 Kerr Cir, Dallas, TX 75244, USA

6. Latitude: 33.011301, Longitude: -96.859813
   Address: 3440 Rosemeade Pkwy, Carrollton, TX 75007, USA

7. Latitude: 32.999692, Longitude: -96.795609
   Address: 18260 Preston Rd #110, Dallas, TX 75252, USA

8. Latitude: 33.021879, Longitude: -96.873324
   Address: 2044 Oakbluff Dr, Carrollton, TX 75007, USA

9. Latitude: 33.029926, Longitude: -96.820631
   Address: 5909 Crownover Ct, Plano, TX 75093, USA


In [14]:
addison_map